You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# Build a Crew to Tailor Job Applications

If you are running this notebook on your own machine, you may need to install the required libraries:

- `crewai`: A library for creating and managing AI agents and tasks.
- `langchain`: A library for building applications with language models.
- `langchain_community`: Community-contributed extensions for LangChain.
- `openai`: OpenAI's API client library.
- `boto3`: The Amazon Web Services (AWS) SDK for Python.
- `python-docx`: A library for creating, modifying, and extracting information from Microsoft Word documents.
- `langchain_aws`: AWS-specific extensions for LangChain.
- `crewai_tools`: Tools for enhancing the functionality of CrewAI.
- `anthropic`: A library for interacting with Anthropic's language models.
- `litellm`: A lightweight library for interacting with various language models.


In [1]:
# These libraries can be installed only once after creating the Python virtual environment (venv).
# Uncomment the lines below to install the libraries.

#%pip install crewai

#%pip install docx2txt

#%pip install --upgrade langchain langchain-community 

#%pip install openai==1.75.0 

#%pip install boto3

#%pip install python-docx

#%pip install langchain_aws

#%pip install crewai_tools 

#%pip install langchain_community

#%pip install anthropic 

#%pip litellm langchain_community

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Activating logging for debugging
import logging
boto3_logger = logging.getLogger("boto3")
boto3_logger.setLevel(logging.DEBUG)

botocore_logger = logging.getLogger("botocore")
botocore_logger.setLevel(logging.DEBUG)

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew, LLM

- As an LLM for your agents, you'll be using Haiku via Bedrock. This setup leverages the Bedrock platform to access the Haiku model, which will be used by the agents to perform their tasks effectively.

In [5]:
# Setting up the Serper API Key
import sys
import os
from utils import get_serper_api_key, get_resume_file_path, pretty_print_result

# Print the environment variables to validate
print("SERPER_API_KEY: ", os.environ["SERPER_API_KEY"])

# Assuming your 'resume' folder is within the current working directory
file_path = os.environ["RESUME_FILE_PATH"]
print("Resume Filepath:", file_path)

In [6]:
## Initialize a language model using the Amazon Bedrock. You can experiment with different models by changing the model parameter.

from langchain_community.llms import Bedrock
 
llm = LLM(model="bedrock/amazon.titan-text-lite-v1", temperature=0.6)

In [7]:
import crewai_tools
from crewai_tools import DOCXSearchTool

# CrewAI Tools

In [8]:
# This code initializes a semantic search tool using Amazon Bedrock as the LLM provider and embedder.
semantic_search_resume = DOCXSearchTool(
    docx=file_path,
    config=dict(
        llm=dict(
            provider="aws_bedrock",  # Using Amazon Bedrock as the LLM provider
            config=dict(
                model="amazon.titan-text-lite-v1",  # Specify the Bedrock-supported model
                temperature=0.6,
                # top_p=1,
                # stream=True,
            ),
        ),
        embedder=dict(
            provider="aws_bedrock",  # Using Amazon Bedrock for embeddings
            config=dict(
                model="amazon.titan-embed-text-v2:0",  # Bedrock-supported embedding model
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

In [9]:
# Import necessary tools from crewai_tools module and initialize them
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, FileReadTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path)

In [10]:
# This code reads a .docx file and displays its content as a Markdown preview in a Jupyter notebook.

from docx import Document

def display_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])  # Extract text from all paragraphs
    return text

#file_path = "resume/sample_resume.docx"  # Upload your resume or use a sample 
resume_text = display_docx(file_path)

from IPython.display import display, Markdown
display(Markdown(f"### Resume Preview\n\n{resume_text}"))

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [11]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [12]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [13]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [14]:
# Agent 4: Interview Coach
interview_preparer = Agent(
    role="Engineering Interview Coach",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

In [15]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=False
)

In [16]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal website "
        "({personal_website}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=False
)

In [17]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="Tailored_Resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist,
    async_execution=False
)

In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential questions and answers based on the profile and requirements of the position. "
        "Use tools to generate questions and answers or potential discussion points that are relevant "
        "to the candidate profile being sought for the position."
Be sure to use these questions and discussion points to
help the candidate highlight the main points of their resume
and how they fit into the position."
    ),
    expected_output=(
        "A document containing key questions and answers to prepair candidates for job interviews, including "
        "talking points that the candidate should prepare for the initial interview."
    ),
    output_file="Interview_Preparation.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer,
    async_execution=False
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.

In [19]:
## USE THIS FOR BEDROCK
#from langchain_community.llms import Bedrock
 
manager_llm = LLM(model="bedrock/amazon.titan-text-lite-v1", temperature=0.6)

In [20]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],
    llm=manager_llm,
    verbose=True,
    debug=True
)

## Running the Crew

In [21]:
job_application_inputs = {
    'job_posting_url': 'https://openai.com/careers/software-engineer-backend/',
    'github_url': 'https://github.com/lessandro-mendes-alhadas',
    'personal_website': 'https://www.linkedin.com/in/lessandroalhadas/'
}

In [22]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

- Display the results of your execution as markdown in the notebook.

In [23]:
from IPython.display import Markdown
Markdown(result.raw)

In [24]:
#Let's estimate cost. You can see pricing at: https://aws.amazon.com/bedrock/pricing/

import pandas as pd

costs = (0.8 * (job_application_crew.usage_metrics.prompt_tokens) + 3.2*(job_application_crew.usage_metrics.completion_tokens) )/ 1000000
print(f"Total costs: ${costs:.4f}")


# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([job_application_crew.usage_metrics.dict()])
df_usage_metrics